<a href="https://colab.research.google.com/github/MichaelCVelez/AIinHealthcare/blob/main/MIMICVisualizations.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Setup

First pip install some required packages. I will be using altair, pygal and bokeh for my visualizations.

In [3]:
!pip install altair_viewer
!pip install pygal
!pip install cairosvg

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 844.5/844.5 kB 12.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.5/129.5 kB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.2/43.2 kB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 2.5 MB/s eta 0:00:00


In [12]:
# Import necessary libraries
import pandas as pd
from datetime import datetime

# Visualization libraries
import altair as alt
import pygal

# Bokeh imports
from bokeh.io import output_notebook, show
from bokeh.models import ColumnDataSource
from bokeh.transform import factor_cmap
from bokeh.palettes import Spectral10
from bokeh.plotting import figure

# Colab file upload utility
from google.colab import files

# Hide FutureWarnings
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

# File Uploads

Save the table data using file upload.

In [5]:
# Load PATIENTS data
d1 = files.upload()
patients = pd.read_csv('PATIENTS.csv')

Saving PATIENTS.csv to PATIENTS.csv


In [6]:
# Load D_ITEMS Dictionary
d2 = files.upload()
d_items = pd.read_csv('D_ITEMS.csv')

Saving D_ITEMS.csv to D_ITEMS.csv


In [7]:
# Load PRESCRIPTIONS Dictionary
d3 = files.upload()
prescriptions = pd.read_csv('PRESCRIPTIONS.csv', on_bad_lines='skip')

Saving PRESCRIPTIONS.csv to PRESCRIPTIONS.csv


In [8]:
# Load OUTPUTEVENTS Dictionary
d4 = files.upload()
outputevents = pd.read_csv('OUTPUTEVENTS.csv')

Saving OUTPUTEVENTS.csv to OUTPUTEVENTS.csv


In [9]:
# Load DRGCODES Dictionary
d5 = files.upload()
drgcodes = pd.read_csv('DRGCODES.csv')

Saving DRGCODES.csv to DRGCODES.csv


In [10]:
# Load PROCEDUREEVENTS_MV Dictionary
d6 = files.upload()
procedureevents_mv = pd.read_csv('PROCEDUREEVENTS_MV.csv')

Saving PROCEDUREEVENTS_MV.csv to PROCEDUREEVENTS_MV.csv


# Altair


In [19]:
# Enable Altair renderer for Colab
alt.renderers.enable('colab')

# Create function calculate_age() which inputs DOB and DOD and returns age
def calculate_age(dob, dod):
    dob_date = datetime.strptime(dob, "%Y-%m-%d %H:%M:%S")
    dod_date = datetime.strptime(dod, "%Y-%m-%d %H:%M:%S")
    return (dod_date - dob_date).days // 365

# Create a new DataFrame 'df_patients' from 'patients'
df_patients = patients.copy()

# Apply calculate_age function to create "age" column
df_patients["age"] = df_patients.apply(lambda row: calculate_age(row["dob"], row["dod"]), axis=1)

# Filter out rows with age greater than or equal to 120
df_filtered = df_patients[df_patients["age"] < 120].copy()

# Create age bins and labels (e.g., 0-10, 10-20, etc.)
bins = [0, 10, 20, 30, 40, 50, 60, 70, 80, 90, 100, 120]
labels = ['0-10', '10-20', '20-30', '30-40', '40-50', '50-60', '60-70', '70-80', '80-90', '90-100', '100-120']

# Create age_group column and assign by age using bins and labels
df_filtered['age_group'] = pd.cut(df_filtered['age'], bins=bins, labels=labels)

# Merge 'prescriptions' and 'df_filtered' on 'subject_id'
df_merged = pd.merge(prescriptions, df_filtered[['subject_id', 'age', 'age_group']], on='subject_id', how='left')

# Filter to include top 10 most prescribed drugs and only their data
top_drugs = df_merged['drug'].value_counts().nlargest(10).index
top_drugs_data = df_merged[df_merged['drug'].isin(top_drugs)]

# Remove rows where 'age_group' is null (invalid data)
top_drugs_data = top_drugs_data[top_drugs_data['age_group'].notnull()]

# Create Altair chart and display
chart = alt.Chart(top_drugs_data).mark_bar().encode(
    x=alt.X('drug', sort='-y', title='Drug Name'),
    y=alt.Y('count()', title='Number of Prescriptions'),
    color='age_group',
    tooltip=['drug', 'count()', 'age_group']
).properties(
    title="Top 10 Prescribed Drugs by Age Group",
    width=600,
    height=400
).interactive()

chart

alt.Chart(...)

In [18]:
# calculate total value (fluid output volume) for each fluid type and ICU stay
df_outputevents = outputevents.copy()

top_10_data_grouped = (
    df_outputevents
    .merge(d_items[['itemid', 'label']], on='itemid', how='left')
    .groupby(['label', 'icustay_id'])['value'].sum().reset_index()
    .pipe(lambda df: df[df['label'].isin(
        df.groupby('label')['value'].sum().nlargest(10).index)])
)

# Create and display Altair chart colored by fluid types
chart = alt.Chart(top_10_data_grouped).mark_bar().encode(
    x=alt.X('sum(value):Q', title='Total Value', axis=alt.Axis(format=',.0f')),
    y=alt.Y('label:N', sort='-x', title='Fluid Type'),
    color='label:N',
    tooltip=['label:N', 'sum(value):Q']
).properties(
    title='Top 10 Fluid Outputs by Fluid Type',
    width=600,
    height=400
).interactive()

chart

alt.Chart(...)

In [26]:
# Merge ‘df_outputevents’ with ‘d_items’ on ‘itemid’ and ‘df_patients‘ on ‘subject_id’.
# Then add new columns 'time', 'date' and 'age_group' using 'charttime' and 'age' columns
data = (
    df_outputevents
    .merge(d_items[['itemid', 'label']], on='itemid')
    .merge(df_patients[['subject_id', 'age']], on='subject_id')
    .assign(
        time=lambda df: pd.to_datetime(df['charttime']),
        date=lambda df: df['time'].dt.strftime('%Y-%m-%d'),
        age_group=lambda df: pd.cut(df['age'], bins=bins).astype(str)
    )
)

# Aggregate the data by 'date', 'age_group', and 'label' to get the total fluid output for each combination.
daily_output = data.groupby(['date', 'age_group', 'label'])['value'].sum().reset_index()

# Create and display interactive Altair line chart where the x-axis is the date and the y-axis is the total fluid output colored by different age groups
chart = alt.Chart(daily_output).mark_line(strokeWidth=3).encode(
    x=alt.X('date:T', axis=alt.Axis(format='%m/%d/%Y', title='Date')),
    y='value:Q',
    color='age_group:N',
    tooltip=['date:T', 'age_group:N', 'label:N', 'value:Q']
).properties(
    title='Fluid Output Distribution Over Time by Age Group',
    width=600,
    height=400
).interactive()

chart

alt.Chart(...)

# Pygal


In [ ]:
# Imported theme for personalized style
from pygal.style import LightSolarizedStyle

# Merge the drgcodes table with the patients table on subject_id using inner join
drgcodes_with_gender = drgcodes.merge(patients[['subject_id', 'gender']], on='subject_id', how='inner')

# Split the merged data by male and female
male_drgcodes = drgcodes_with_gender[drgcodes_with_gender['gender'] == 'M']
female_drgcodes = drgcodes_with_gender[drgcodes_with_gender['gender'] == 'F']

# Count the occurrences of each DRG code for male and female
male_drgcode_counts = male_drgcodes['drg_code'].value_counts()
female_drgcode_counts = female_drgcodes['drg_code'].value_counts()

# Get the top 5 most frequent DRG codes for both male and female
top_5_male_drgcodes = male_drgcode_counts.head(5)
top_5_female_drgcodes = female_drgcode_counts.head(5)

# Create a dictionary from DRG code to its description
drgcode_to_description = dict(zip(drgcodes['drg_code'], drgcodes['description']))

# Get a set of the top 5 DRG codes for both genders combined
top_5_combined_drgcodes = set(top_5_male_drgcodes.index).union(set(top_5_female_drgcodes.index))

# Create a list of descriptions for the combined top DRG codes
top_5_descriptions = [drgcode_to_description.get(drg_code, drg_code) for drg_code in top_5_combined_drgcodes]

# Prepare the counts for male and female for each DRG code
male_counts = [top_5_male_drgcodes.get(drg_code, 0) for drg_code in top_5_combined_drgcodes]
female_counts = [top_5_female_drgcodes.get(drg_code, 0) for drg_code in top_5_combined_drgcodes]

# Create a StackedBar chart using Pygal
bar_chart = pygal.StackedBar(width=2500, height=500, style=LightSolarizedStyle)
bar_chart.title = 'Top 5 Most Frequent DRG Codes by Gender'
bar_chart.show_legend = True
bar_chart.legend_at_bottom = True
bar_chart.x_labels = ['Male', 'Female']

# Add DRG code data to bar chart stacked by DRG code for both male and female.
for i, drg_code in enumerate(top_5_combined_drgcodes):
    description = top_5_descriptions[i]
    bar_chart.add(description, [male_counts[i], female_counts[i]])

# Display the chart
bar_chart

# Bokeh

In [99]:
# Import necessary Bokeh libraries
from bokeh.transform import factor_cmap
from bokeh.io import output_notebook, show
from bokeh.plotting import figure
from bokeh.models import ColumnDataSource
from bokeh.palettes import Spectral5

# Enable Bokeh to output to notebook
output_notebook()

# Merge the 'proceduresevents_mv' table with 'patients' using inner join on subject_id'
procedures_with_gender = procedureevents_mv.merge(patients[['subject_id', 'gender']], on='subject_id', how='inner')

# Split the data by both male and female
male_procedures = procedures_with_gender[procedures_with_gender['gender'] == 'M']
female_procedures = procedures_with_gender[procedures_with_gender['gender'] == 'F']

# Get the top 5 most common procedures for both male and female
top_5_male_procedures = male_procedures['itemid'].value_counts().head(5).reset_index()
top_5_male_procedures.columns = ['itemid', 'count']

top_5_female_procedures = female_procedures['itemid'].value_counts().head(5).reset_index()
top_5_female_procedures.columns = ['itemid', 'count']

# Merge the top 5 procedures with 'd_items' to get the procedure labels
top_5_male_procedures = top_5_male_procedures.merge(d_items[['itemid', 'label']], on='itemid', how='left')
top_5_female_procedures = top_5_female_procedures.merge(d_items[['itemid', 'label']], on='itemid', how='left')

# Convert data to lists for the stacked bar chart
procedure_labels = top_5_male_procedures['label'].tolist()
male_counts = top_5_male_procedures['count'].tolist()
female_counts = top_5_female_procedures['count'].tolist()

# Create a DataFrame formatted for Bokeh
data = {
    'gender': ['Male', 'Female'],
    procedure_labels[0]: [male_counts[0], female_counts[0]],
    procedure_labels[1]: [male_counts[1], female_counts[1]],
    procedure_labels[2]: [male_counts[2], female_counts[2]],
    procedure_labels[3]: [male_counts[3], female_counts[3]],
    procedure_labels[4]: [male_counts[4], female_counts[4]]
}

# Convert the data to ColumnDataSource
source = ColumnDataSource(data)

# Create plot for stacked bar chart
plot = figure(x_range=['Male', 'Female'], height=700, width=600, title="Top 5 Most Common Procedures by Gender",
           toolbar_location=None, tools="hover", tooltips="$name: @$name")

# Add stacked bars to the plot
plot.vbar_stack(procedure_labels, x='gender', width=0.5, color=Spectral5, source=source,
             legend_label=procedure_labels)

# Customize the plot
plot.y_range.start = 0
plot.xaxis.axis_label = "Gender"
plot.yaxis.axis_label = "Count"
plot.xgrid.grid_line_color = None
plot.legend.location = "top_right"
plot.legend.orientation = "vertical"

# Display the plot
show(plot)